# Set-up

## Imports

In [1]:
%load_ext autoreload
%autoreload 2

import glob
import ipdb
import numpy as np
import os
import pandas as pd
import time

from AlienTask import Task
from shared_aliens import get_alien_paths, get_alien_initial_q, get_summary_initial_learn#, #update_Qs_sim, get_alien_paths, get_alien_initial_q,\
#     get_summary_initial_learn, get_summary_cloudy, simulate_competition_phase, simulate_rainbow_phase, get_summary_rainbow,\
#     read_in_human_data, get_chosen_TS
from SimulateAliensJupyterVersionFunctions import *

## Switches

In [2]:
# Define things
do_isomap = False
do_save_selected_agents = True
if do_isomap:
    from sklearn import manifold, decomposition, preprocessing

# Which model will be simulated?
model_name = 'Bayes'
# models = ['hier', 'Bayes', 'flat']
n_sim_per_subj, n_subj = 1, 26  # n_sim_per_sub = 20, n_subj = 31 (version3.1)  # TODO should be 1, 31-x (1 sim per person; exclude excluded subjects)
n_sim = n_sim_per_subj * n_subj
n_actions, n_aliens, n_seasons, n_TS = 3, 4, 3, 3
alien_initial_Q = get_alien_initial_q(model_name)

## Paths

In [3]:
# Data paths
## Where is human experimental data stored?
human_data_path = get_alien_paths()["human data prepr"]

## Where will simulated summaries be saved or read in?
summary_save_dir = os.path.join(get_alien_paths()['fitting results'], 'Summaries')
full_summary_save_dir = os.path.join(summary_save_dir, 'FullSimulations')

## Where will plots be saved that we create here?
plot_save_dir = os.path.join(summary_save_dir, 'plots')  # 'C:/Users/maria/MEGAsync/Berkeley/TaskSets/paperplots/'

## Initialize trial object

In [4]:
# Create task, get numbers of trials for each phase
task = Task(n_subj)
n_trials, _, _, _, _ = task.get_trial_sequence(get_alien_paths()["human data prepr"],
                                         n_subj, n_sim_per_subj, range(n_subj),
                                         phases=("1InitialLearning", "2CloudySeason"))

n_trials_ = {'1InitialLearn': np.sum(task.phase == '1InitialLearning'),
             '2CloudySeason': np.sum(task.phase == '2CloudySeason'),
             '4RainbowSeason': 4 * n_aliens,
             '5Competition': 3}

trials = {'1InitialLearn': range(n_trials_['1InitialLearn']),
          '2CloudySeason': range(n_trials_['1InitialLearn'],
                                 n_trials_['1InitialLearn'] + n_trials_['2CloudySeason'])}

# Create summaries

In [5]:
save_frequency = 20
print_frequency = 20
n_frames = 100
n_sims_per_frame = 1000

# Get summaries for different parameters
new_time = time.time()

for frame_i in range(n_frames):
    summaries = pd.DataFrame()

    for sim_i in range(n_sims_per_frame):
        params = np.random.rand(len(param_names))
        new_summary = get_summary(
            task, model_name, params, param_ranges,
            n_sim, trials, n_trials, n_trials_, alien_initial_Q, summary_or_fulldata='summary')
        summaries = pd.concat([summaries, new_summary], axis=1)

        if (sim_i % print_frequency) == 0:

            # Print progress
            print("\n\tIteration {}".format(sim_i))
            old_time = new_time
            new_time = time.time()
            diff = new_time - old_time
            print("Time passed: {} seconds".format(np.round(diff)))

        if ((sim_i % save_frequency) == 0) or (sim_i == n_sims_per_frame - 1):

            # Save summaries to disk
            save_path = os.path.join(summary_save_dir, model_name + '_summaries_{}TEST.csv'.format(frame_i))
            save_summaries = summaries.T
            save_summaries['model'] = model_name
            save_summaries.to_csv(save_path)
            print("Saving summaries to {}".format(save_path))


	Iteration 0
Time passed: 4.0 seconds
Saving summaries to C:/Users/maria/MEGAsync/Berkeley/TaskSets/models\Summaries\Bayes_summaries_0TEST.csv


KeyboardInterrupt: 